In [1]:
# import 
import numpy as np
import random
from proglearn.sims import generate_gaussian_parity
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
import sklearn.ensemble
from sklearn import metrics 
import math
from scipy.stats import ttest_ind
from scipy.spatial import distance
import random
import pandas as pd

In [2]:
# number of samples
n_samples = 100

# generate xor 
X_xor, y_xor = generate_gaussian_parity(n_samples, angle_params=0)

# generate rxor, 45 degrees
X_rxor, y_rxor = generate_gaussian_parity(n_samples, angle_params=np.pi/4)

In [3]:
# we want to pick 70 samples from xor/rxor to train trees so we need to first subset each into arrays with only xor_0/1 and rxor_0/1
X_xor_0 = X_xor[np.where(y_xor == 0)]
X_xor_1 = X_xor[np.where(y_xor == 1)]

X_rxor_0 = X_rxor[np.where(y_rxor == 0)]
X_rxor_1 = X_rxor[np.where(y_rxor == 1)]

# we can concat the first 35 samples from each pair to use to tatal 70 samples for training and 30 for predict proba
X_xor_train = np.concatenate((X_xor_0[0:35], X_xor_1[0:35]))
y_xor_train = np.concatenate((np.zeros(35), np.ones(35)))

# repeat for rxor
X_rxor_train = np.concatenate((X_rxor_0[0:35], X_rxor_1[0:35]))
y_rxor_train = np.concatenate((np.zeros(35), np.ones(35)))

In [4]:
len(X_xor_train), len(X_xor_train)

(70, 70)

In [5]:
# init the rf's
# xor rf
clf_xor = sklearn.ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=int(n_samples/7))

# rxor rf
clf_rxor = sklearn.ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=int(n_samples/7))

In [6]:
# train rfs
# fit the model using the train data 
clf_xor.fit(X_xor_train, y_xor_train)

# fit rxor model
clf_rxor.fit(X_rxor_train, y_rxor_train)

RandomForestClassifier(min_samples_leaf=14, n_estimators=10)

In [13]:
# concat the test samples from xor and rxor (30 from each), 60 total test samples
X_xor_rxor_test = np.concatenate((X_xor_0[35:], X_rxor_0[35:], X_xor_1[35:], X_rxor_1[35:]))
y_xor_rxor_test = np.concatenate((np.zeros(30), np.ones(30)))

# confirm that there's 60 in each
len(X_xor_rxor_test), len(y_xor_rxor_test)

(60, 60)

In [21]:
# predict proba on the new test data with both rfs
# xor rf
xor_rxor_test_xorRF_probas = clf_xor.predict_proba(X_xor_rxor_test)

# rxor rf
xor_rxor_test_rxorRF_probas = clf_rxor.predict_proba(X_xor_rxor_test)

In [25]:
def calcL2(xorRF_probas, rxorRF_probas):
    '''
    Returns L2 distance between 2 outputs from clf.predict_proba().
    '''
    # lists to store % label 0 since we only need one of the probas to calc L2
    xors = []
    rxors = []
    
    # iterate through the passed probas to store them in our lists
    for xor_proba, rxor_proba in zip(xorRF_probas, rxorRF_probas):
        xors.append(xor_proba[0])
        rxors.append(rxor_proba[0])
        
    return distance.euclidean(xors, rxors)

In [28]:
# calc the l2 distance between the probas from xor and rxor rfs
d1 = calcL2(xor_rxor_test_xorRF_probas, xor_rxor_test_rxorRF_probas)
d1

2.1260755784863266

In [30]:
# concat all xor and rxor samples (100+100=200)
X_xor_rxor_all = np.concatenate((X_xor, X_rxor))
y_xor_rxor_all = np.concatenate((y_xor, y_rxor))

In [35]:
random_idxs = random.sample(range(200), 200)

In [39]:
len(random_idxs[0:100]), len(random_idxs[100:])

(100, 100)

In [40]:
# subsample 100 samples twice randomly, call one xor and the other rxor
X_xor_new = X_xor_rxor_all[random_idxs[0:100]]
y_xor_new = y_xor_rxor_all[random_idxs[0:100]]

X_rxor_new = X_xor_rxor_all[random_idxs[100:]]
y_rxor_new = y_xor_rxor_all[random_idxs[100:]]

In [43]:
len(X_xor_new), len(y_xor_new), len(X_rxor_new), len(y_rxor_new)

(100, 100, 100, 100)

In [48]:
# subsample 70 from each and call one xor train and one rxor train
# since we randomly took 100 the pool of 200 samples we should just be able to take the first 70 samples 
X_xor_new_train = X_xor_new[0:70]
y_xor_new_train = y_xor_new[0:70]

X_rxor_new_train = X_rxor_new[0:70]
y_rxor_new_train = y_rxor_new[0:70]

In [49]:
# train a new forest 
# init the rf's
# xor rf
clf_xor_new = sklearn.ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=int(n_samples/7))
clf_xor_new.fit(X_xor_new_train, y_xor_new_train)

# rxor rf
clf_rxor_new = sklearn.ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=int(n_samples/7))
clf_rxor_new.fit(X_rxor_new_train, y_rxor_new_train)

RandomForestClassifier(min_samples_leaf=14, n_estimators=10)

In [50]:
# take the remaing 30 and call those test
X_xor_new_test = X_xor_new[70:]
y_xor_new_test = y_xor_new[70:]

X_rxor_new_test = X_rxor_new[70:]
y_rxor_new_test = y_rxor_new[70:]

In [51]:
len(X_xor_new_test), len(X_rxor_new_test)

(30, 30)

In [52]:
# concat our new samples
X_xor_rxor_new_test = np.concatenate((X_xor_new_test, X_rxor_new_test))
y_xor_rxor_new_test = np.concatenate((y_xor_new_test, y_rxor_new_test))

In [53]:
# predict proba using the original xor and rxor rf's and calc l2
# new xor rf
xor_rxor_new_test_xorRF_probas = clf_xor_new.predict_proba(X_xor_rxor_new_test)

# new rxor rf
xor_rxor_new_test_rxorRF_probas = clf_rxor_new.predict_proba(X_xor_rxor_new_test)

In [54]:
# calc l2 for our new data
d2 = calcL2(xor_rxor_new_test_xorRF_probas, xor_rxor_new_test_rxorRF_probas)

In [55]:
d2

1.0233096510361408

d2 > d1, so the distributions are different 